In [1]:
import tensorflow as tf

from configuration import Config
from core.efficientdet import EfficientDet, PostProcessing


def print_model_summary(network):
    sample_inputs = tf.random.normal(shape=(Config.batch_size, Config.get_image_size()[0], Config.get_image_size()[1], Config.image_channels))
    sample_outputs = network(sample_inputs, training=True)
    network.summary()

#모델 구조
model = EfficientDet()
print_model_summary(model)

#weight 불러오기
load_weights_from_epoch = Config.load_weights_from_epoch
model.load_weights(filepath=Config.save_model_dir+"epoch-{}".format(load_weights_from_epoch))

2021-10-12 22:21:16.719013: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-12 22:21:17.439757: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21725 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:86:00.0, compute capability: 7.5
2021-10-12 22:21:18.564513: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2021-10-12 22:21:19.184347: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Model: "efficient_det"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficient_net (EfficientNet) multiple                  6771296   
_________________________________________________________________
bi_fpn (BiFPN)               multiple                  129126    
_________________________________________________________________
box_class_predict (BoxClassP multiple                  249069    
Total params: 7,149,491
Trainable params: 7,106,515
Non-trainable params: 42,976
_________________________________________________________________


In [4]:
dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'\\*')


# 여기 부분, 기존 코드의 data_preprocesssing 과 동일하게 맞추기
def representative_data_gen():
  dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'\\*')
  for i in range(100):
    image = next(iter(dataset_list))
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (512,512))
    # image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
# Model has only one input so each data point has one element
    yield [image]
    

#     #### 수정좀 해보기
# def representative_data_gen():
#   dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'\\*')
#   for i in range(100):
#     image = next(iter(dataset_list))
#     image = tf.io.read_file(image)
#     image = tf.io.decode_jpeg(image, channels=3)
#     image = tf.image.resize(image, (512,512))
#     # image = tf.cast(image / 255., tf.float32)
#     image = tf.expand_dims(image, 0)
# # Model has only one input so each data point has one element
#     yield [image]

InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: ./data/fire_smoke/train/JPEGImages/\\*'

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.experimental_new_quantizer = True
converter.inference_input_type = tf.uint8
#converter.inference_output_type = tf.uint8
converter.inference_output_type = tf.float32

tflite_model_quant = converter.convert()

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
for item in interpreter.get_tensor_details():
    for key in item.keys():
        print("%s : %s" % (key, item[key]))
    print("")

In [ ]:
with open('./tflite_model_quant.tflite', 'wb') as f:
  f.write(tflite_model_quant)

In [ ]:
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
print(input_details)
print(output_details)

In [8]:
model_path = "C:/Users/yutan/Desktop/EfficientDet_TensorFlow2/tflite_model_quant.tflite"

def load_model(path):  
    interpreter = tf.lite.Interpreter(model_path = path)
    return interpreter

def prediction(model, data):
    interpreter = model
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]
    input_data = np.array(data, dtype = np.uint8)
    interpreter.set_tensor(input_details['index'], input_data)
    interpreter.invoke()
    output= interpreter.get_tensor(output_details['index'])
    return output

In [9]:
dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'\\*')

image = next(iter(dataset_list))
image = tf.io.read_file(image)
image = tf.io.decode_jpeg(image, channels=3)
image = tf.image.resize(image, (512,512))
# image = tf.cast(image / 255., tf.float32)
image = tf.expand_dims(image, 0)
print(image.shape)

(1, 512, 512, 3)


In [10]:
import numpy as np

model = load_model(model_path)
output = prediction(model, image)

In [12]:
print(output.shape)
print(output)

(1, 49104, 5)
[[[ 0.00297724 -0.03572687  0.         -0.00297724  0.49124447]
  [-0.01786344 -0.0833627   0.02084067 -0.00893172  0.5091079 ]
  [ 0.         -0.04168135 -0.07443098  0.01786344  0.48826724]
  ...
  [-0.00595448  0.03274963  0.05656755  0.03572687  0.49124447]
  [-0.03274963  0.04168135 -0.05061307  0.00893172  0.5388803 ]
  [-0.04763583 -0.04763583 -0.00595448  0.00297724  0.5001762 ]]]
